In [8]:
import pandas as pd
df_paris = pd.read_csv("CSV/paris.csv")
df_lyon = pd.read_csv("CSV/lyon.csv")
df_Bordeaux = pd.read_csv("CSV/bordeaux.csv")
df_Pays_Basque = pd.read_csv("CSV/paysbasque.csv")

In [11]:
df_paris.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license'],
      dtype='object')

In [ ]:
df_paris['']